In [4]:
import os
import time
import csv
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import save_model, load_model
from keras.callbacks import EarlyStopping
import plotly.express as px
import plotly.graph_objs as go
import joblib

# Erstelle den Ordner zum Speichern der Modelle, falls nicht bereits vorhanden
os.makedirs('./models', exist_ok=True)


In [5]:
# Lade die historischen Daten einer Aktie (z.B. Apple)
symbol = "AAPL"  # Apple-Aktien
start_date = "2000-01-01"  # Startdatum für die Daten
end_date = "2024-10-10"    # Enddatum für die Daten

# CSV-Datei für die Ergebnisse
csv_filename = f'./results/hyperparameter_search_results_{start_date}_{end_date}.csv'
os.makedirs('./results', exist_ok=True)

# Daten herunterladen von Yahoo Finance
df = yf.download(symbol, start=start_date, end=end_date)

# Überprüfen der Daten
print(df.head())


[*********************100%***********************]  1 of 1 completed

                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2000-01-03  0.936384  1.004464  0.907924  0.999442   0.844004  535796800
2000-01-04  0.966518  0.987723  0.903460  0.915179   0.772846  512377600
2000-01-05  0.926339  0.987165  0.919643  0.928571   0.784155  778321600
2000-01-06  0.947545  0.955357  0.848214  0.848214   0.716296  767972800
2000-01-07  0.861607  0.901786  0.852679  0.888393   0.750226  460734400


In [6]:
# Daten aus den relevanten Spalten entnehmen
features = df[['Open', 'High', 'Low', 'Adj Close', 'Volume']]
target = df['Close']

# MinMaxScaler initialisieren und die Features skalieren
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

scaled_features = feature_scaler.fit_transform(features)
scaled_target = target_scaler.fit_transform(target.values.reshape(-1, 1))

joblib.dump(feature_scaler, './scaler/feature_scaler.pkl')
joblib.dump(target_scaler, './scaler/target_scaler.pkl')

['./scaler/target_scaler.pkl']

In [4]:
val_size= 0.2
# Definiere die möglichen Werte für die Hyperparameter
hyperparameters = {
    'windowsize': [1, 2, 5, 10, 15, 20, 30, 60],
    'batchsize': [1, 5, 14, 30, 64, 128, 256, 512],
    'epochs': [30],
    'units': [1, 2, 5, 6, 8, 12, 16, 32, 64, 128],
    'lstm_layers': [1, 2, 3]
}

In [5]:
# Prüfe, ob das Modell bereits existiert
def model_exists(window_size, batch_size, epochs, units, lstm_layers):
    if os.path.exists(csv_filename):
        with open(csv_filename, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[1:6] == [str(window_size), str(batch_size), str(epochs), str(units), str(lstm_layers)]:
                    return True
    return False

# Schreibe oder appende die Ergebnisse in die CSV
def write_to_csv(model_name, window_size, batch_size, epochs, units, lstm_layers, mse, mae, r2):
    file_exists = os.path.isfile(csv_filename)
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Header hinzufügen, wenn die Datei neu erstellt wird
            writer.writerow(['Model Name', 'Window Size', 'Batch Size', 'Epochs', 'Units', 'LSTM Layers', 'MSE', 'MAE', 'R2'])
        writer.writerow([model_name, window_size, batch_size, epochs, units, lstm_layers, mse, mae, r2])

def create_windowed_data(features, target, window_size=60):
    X, y = [], []
    for i in range(window_size, len(features)):
        X.append(features[i-window_size:i])
        y.append(target[i])
    return np.array(X), np.array(y)

# Zufällige Suche über die Hyperparameter
def random_search(n_iter=10):
    results = []

    for _ in range(n_iter):
        # Zufällige Auswahl von Hyperparametern
        window_size = np.random.choice(hyperparameters['windowsize'])
        batch_size = np.random.choice(hyperparameters['batchsize'])
        epochs = np.random.choice(hyperparameters['epochs'])
        units = int(np.random.choice(hyperparameters['units']))
        lstm_layers = np.random.choice(hyperparameters['lstm_layers'])

        print(f"Iteration: {_}, window_size={window_size}, batch_size={batch_size}, epochs={epochs}, units={units}, lstm_layers={lstm_layers}")

        # Prüfe, ob das Modell mit diesen Parametern bereits existiert
        if model_exists(window_size, batch_size, epochs, units, lstm_layers):
            print(f"Modell mit den Parametern (ws={window_size}, bs={batch_size}, ep={epochs}, units={units}, layers={lstm_layers}) existiert bereits. Überspringe...")
            continue

        # Daten vorbereiten mit aktuellem window_size
        X, y = create_windowed_data(scaled_features, scaled_target, window_size=window_size)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, random_state=42)

        # Modell erstellen
        model = Sequential()

        # Ersten LSTM Layer hinzufügen
        model.add(LSTM(units=units, return_sequences=True if lstm_layers > 1 else False, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(0.2))

        # Weitere LSTM Layer falls vorhanden
        for layer in range(1, lstm_layers):
            model.add(LSTM(units=units, return_sequences=False if layer == lstm_layers - 1 else True))
            model.add(Dropout(0.2))

        # Dense-Schicht hinzufügen
        model.add(Dense(units=1))

        # Modell kompilieren
        model.compile(optimizer='adam', loss='mean_squared_error')
        earlystop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        # Modell trainieren
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=1, callbacks=[earlystop])

        # Vorhersagen auf den Validierungsdaten
        y_pred = model.predict(X_val)

        # Den Skalierungsfaktor rückgängig machen, um die echten Werte wiederherzustellen
        y_val_true = scaler.inverse_transform(y_val.reshape(-1, 1))
        y_pred_rescaled = scaler.inverse_transform(y_pred)

        # Berechne die Metriken
        mse = mean_squared_error(y_val_true, y_pred_rescaled)
        mae = mean_absolute_error(y_val_true, y_pred_rescaled)
        r2 = r2_score(y_val_true, y_pred_rescaled)

        # Ergebnisse ausgeben
        print(f"Validation MSE: {mse}, MAE: {mae}, R2: {r2}")

        # Modellname basierend auf einem Zeitstempel
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        model_name = f"model_{timestamp}_ws{window_size}_bs{batch_size}_epochs{epochs}_units{units}_layers{lstm_layers}.keras"
        save_model(model, f'./models/{model_name}')

        # Ergebnis in die CSV-Datei schreiben
        write_to_csv(model_name, window_size, batch_size, epochs, units, lstm_layers, mse, mae, r2)

        # Ergebnis in die Resultsliste einfügen
        results.append((window_size, batch_size, epochs, units, lstm_layers, mse, mae, r2, model_name))

    # Ergebnisse sortieren nach dem besten MSE
    results = sorted(results, key=lambda x: x[5])  # Sortiere nach MSE
    best_params = results[0]

    print("\nBeste Hyperparameter-Kombination:")
    print(f"window_size={best_params[0]}, batch_size={best_params[1]}, epochs={best_params[2]}, units={best_params[3]}, lstm_layers={best_params[4]}")

    return results

In [6]:
# Beispielhafte Ausführung der Funktion
results = random_search(n_iter=53)

Iteration: 0, window_size=1, batch_size=14, epochs=30, units=16, lstm_layers=1
Epoch 1/30
356/356 [==============================] - 1s 1ms/step - loss: 0.0131 - val_loss: 2.1006e-04
Epoch 2/30
356/356 [==============================] - 0s 505us/step - loss: 0.0018 - val_loss: 1.3897e-04
Epoch 3/30
356/356 [==============================] - 0s 497us/step - loss: 0.0017 - val_loss: 1.2752e-04
Epoch 4/30
356/356 [==============================] - 0s 489us/step - loss: 0.0015 - val_loss: 1.8050e-04
Epoch 5/30
356/356 [==============================] - 0s 495us/step - loss: 0.0013 - val_loss: 7.3828e-05
Epoch 6/30
356/356 [==============================] - 0s 503us/step - loss: 0.0013 - val_loss: 4.7223e-05
Epoch 7/30
356/356 [==============================] - 0s 511us/step - loss: 0.0011 - val_loss: 4.9034e-05
Epoch 8/30
356/356 [==============================] - 0s 536us/step - loss: 0.0011 - val_loss: 5.6299e-05
Epoch 9/30
356/356 [==============================] - 0s 500us/step - loss:

KeyboardInterrupt: 

In [7]:
def plot_predictions_from_models(X_data, y_data, data_type, model_names=None, csv_filename='./results/hyperparameter_search_results.csv', save_fig=True):
    """
    Diese Funktion plotet die Vorhersagen ausgewählter Modelle gegen den echten Kurs.
    :param X_data: Die Eingangsdaten (Features).
    :param y_data: Die echten Zielwerte (z.B. Aktienkurs).
    :param data_type: Typ der Daten ('Train', 'Test').
    :param model_names: Liste von Modellnamen oder einzelner Modellname. Wenn None, werden alle Modelle aus der CSV-Datei verwendet.
    :param csv_filename: Pfad zur CSV-Datei mit Modellinformationen.
    :param save_fig: Wenn True, wird die Figur gespeichert.
    """
    
    # Scaler laden
    scaler = joblib.load('./scaler/scaler.pkl')
    
    # CSV-Datei laden
    df = pd.read_csv(csv_filename)
    
    # Wenn keine Modellnamen übergeben werden, alle Modelle aus der CSV verwenden
    if model_names is None:
        model_names = df['Model Name'].tolist()
    elif isinstance(model_names, str):  # Falls nur ein einzelner Modellname übergeben wird
        model_names = [model_names]

    # Rückskalieren der echten Werte
    y_data_rescaled = scaler.inverse_transform(y_data.reshape(-1, 1))

    # Erstelle einen Plotly-Plot
    fig = go.Figure()

    # Füge die echten Werte hinzu
    fig.add_trace(go.Scatter(y=y_data_rescaled.flatten(), mode='lines', name=f'Echte {data_type} Werte', line=dict(color='black')))

    # Vorhersagen der ausgewählten Modelle plotten
    for model_name in model_names:
        if model_name in df['Model Name'].values:
            # Extrahiere den window_size aus dem Modellnamen (z.B. 'model_20211010-101010_ws30_bs32_...')
            window_size = int(model_name.split('_ws')[1].split('_')[0])

            # Bereite die Daten für das Modell mit der extrahierten window_size vor
            X_windowed = create_windowed_data_for_model(X_data, window_size)

            # Lade das Modell
            model = load_model(f'./models/{model_name}', compile=False)

            # Vorhersagen machen
            y_pred = model.predict(X_windowed)
            y_pred_rescaled = scaler.inverse_transform(y_pred)

            # Modellvorhersage in den Plot einfügen
            fig.add_trace(go.Scatter(y=y_pred_rescaled.flatten(), mode='lines', name=f'{model_name}', line=dict(width=1)))
        else:
            print(f"Modell {model_name} wurde in der CSV-Datei nicht gefunden.")

    # Layout des Plots
    fig.update_layout(
        title=f'Vorhersagen der Modelle auf {data_type}daten',
        xaxis_title='Zeit',
        yaxis_title='Preis',
        showlegend=True
    )

    # Plot anzeigen
    fig.show(renderer='browser')
    
    # Optionales Speichern des Plots
    if save_fig:
        fig.write_html(f"./plots/{time.strftime('%Y%m%d-%H%M%S')}.html")


def create_windowed_data_for_model(X_data, window_size):
    """
    Bereitet die Daten so vor, dass sie für Modelle mit unterschiedlicher window_size verwendet werden können.
    :param X_data: Originale Daten, die vorfenstert werden sollen.
    :param window_size: Die Fenstergröße, die für das Modell verwendet wurde.
    :return: Fensterartige Daten für das Modell.
    """
    X_windowed = []
    for i in range(window_size, len(X_data)):
        X_windowed.append(X_data[i - window_size:i])
    return np.array(X_windowed)


In [5]:
plot_predictions_from_models(scaled_features, scaled_target, data_type='Prediction')

193/193 [==============================] - 1s 2ms/step


In [9]:
plot_predictions_from_models(scaled_features, scaled_target, data_type='Prediction', model_names=['model_20241011-171432_ws1_bs5_epochs30_units64_layers1.keras'], csv_filename=csv_filename)

195/195 [==============================] - 0s 301us/step
